In [2]:
!pip install pandas sqlalchemy ipython-sql

In [4]:
%load_ext sql


In [6]:
from sqlalchemy import create_engine

# Create a connection to a file-based SQLite database
engine = create_engine("sqlite:///fetch_sqlmagic.db")


In [8]:
import pandas as pd

# Load CSV files into pandas DataFrames
users_df = pd.read_csv("C:/Data/Ramya Career/Resume/NumpyNinja/Fetch/FORMATTED/formatted_users.csv")
transactions_df = pd.read_csv("C:/Data/Ramya Career/Resume/NumpyNinja/Fetch/FORMATTED/formatted_transactions.csv")
products_df = pd.read_csv("C:/Data/Ramya Career/Resume/NumpyNinja/Fetch/FORMATTED/formatted_products.csv")


In [10]:
# Save DataFrames as tables in SQLite database
users_df.to_sql("users", con=engine, index=False, if_exists='replace')
transactions_df.to_sql("transactions", con=engine, index=False, if_exists='replace')
products_df.to_sql("products", con=engine, index=False, if_exists='replace')


845552

In [12]:
from sqlalchemy import inspect

# Create an inspector to verify table names
inspector = inspect(engine)
print(inspector.get_table_names())  # Should display ['users', 'transactions', 'products']


['products', 'transactions', 'users']


In [14]:
%sql sqlite:///fetch_sqlmagic.db


In [16]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';


 * sqlite:///fetch_sqlmagic.db
Done.


name
users
transactions
products


In [185]:
%%sql
SELECT * FROM users LIMIT 10;


 * sqlite:///fetch_sqlmagic.db
Done.


id,created_date,birth_date,state,language,gender
5ef3b4f17053ab141787697d,2020-06-24 20:17:54+00:00,2000-08-11 00:00:00+00:00,CA,es-419,female
5ff220d383fcfc12622b96bc,2021-01-03 19:53:55+00:00,2001-09-24 04:00:00+00:00,PA,en,female
6477950aa55bb77a0e27ee10,2023-05-31 18:42:18+00:00,1994-10-28 00:00:00+00:00,FL,es-419,female
658a306e99b40f103b63ccf8,2023-12-26 01:46:22+00:00,None,NC,en,None
653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50+00:00,1972-03-19 00:00:00+00:00,PA,en,female
5fe2b6f3ad416a1265c4ab68,2020-12-23 03:18:11+00:00,1999-10-27 04:00:00+00:00,NY,en,female
651210546816bb4d035b1ead,2023-09-25 22:57:24+00:00,1983-09-25 22:57:25+00:00,FL,es-419,male
642831ea3d4434e63c1936fd,2023-04-01 13:30:18+00:00,1970-02-16 05:00:00+00:00,IN,en,female
63a4c9a1b5f32149b9d82f9e,2022-12-22 21:18:25+00:00,1982-12-22 05:00:00+00:00,NC,en,female
63654b21d02459d8a57a2e2c,2022-11-04 17:25:53+00:00,1992-05-03 04:00:00+00:00,NY,en,non_binary


In [18]:
%%sql
WITH dips_and_salsa AS (
    SELECT p.brand, p.barcode
    FROM products p
    WHERE LOWER(p.category_2) = 'dips & salsa'
    AND p.brand IS NOT NULL
),
brand_sales AS (
    SELECT 
        ds.brand,
        COUNT(DISTINCT t.receipt_id) AS receipt_count,
        SUM(t.sale) AS brand_total_sales
    FROM transactions t
    JOIN dips_and_salsa ds ON t.barcode = ds.barcode
    GROUP BY ds.brand
),
total_category_sales AS (
    SELECT SUM(brand_total_sales) AS total_sales
    FROM brand_sales
)
SELECT 
    bs.brand,
    bs.brand_total_sales,
    bs.receipt_count,
    ROUND((bs.brand_total_sales * 100.0 / tcs.total_sales), 2) AS market_share_percent
FROM brand_sales bs
JOIN total_category_sales tcs ON 1=1
ORDER BY market_share_percent DESC, receipt_count DESC
LIMIT 5;


 * sqlite:///fetch_sqlmagic.db
Done.


brand,brand_total_sales,receipt_count,market_share_percent
TOSTITOS,260.99,36,14.1
GOOD FOODS,118.89,9,6.42
PACE,118.58,24,6.41
MARKETSIDE,103.29,16,5.58
FRITOS,91.73,19,4.96


In [31]:
%%sql
WITH eligible_users AS (
    SELECT 
        id,
        created_date,
        -- Calculate the months since the account was created
        (strftime('%Y', 'now') - strftime('%Y', created_date)) * 12 + 
        (strftime('%m', 'now') - strftime('%m', created_date)) AS months_since_created
    FROM users
),
total_sales_amount AS (
    SELECT p.brand,months_since_created,
    SUM(t.sale) AS total_sales,
   ROW_NUMBER() OVER (ORDER BY SUM(t.sale) DESC) AS rank
    FROM transactions t
    JOIN eligible_users u ON t.user_id = u.id
    JOIN products p on t.barcode = p.barcode
    WHERE months_since_created >= 6
    AND p.brand IS NOT NULL
    GROUP BY p.brand
)
SELECT brand, total_sales
FROM total_sales_amount
WHERE rank <=5
ORDER BY total_sales DESC
        


 * sqlite:///fetch_sqlmagic.db
Done.


brand,total_sales
CVS,72.0
TRIDENT,46.72
DOVE,42.88
COORS LIGHT,34.96
AXE,15.98


In [20]:
%%sql
WITH yearly_user_counts AS (
    -- Step 1: Calculate user count per year
    SELECT 
        STRFTIME('%Y', created_date) AS year,
        COUNT(*) AS user_count
    FROM users
    WHERE created_date IS NOT NULL
    GROUP BY year
    ORDER BY year
)
SELECT 
    year,
    user_count,
    previous_year_count,
    CASE
        WHEN previous_year_count IS NULL THEN NULL
        ELSE ROUND(((user_count - previous_year_count) * 100.0) / previous_year_count, 2)
    END AS yoy_growth_percentage
FROM (
    -- Step 2: Add previous year count only once
    SELECT 
        year,
        user_count,
        LAG(user_count) OVER (ORDER BY year) AS previous_year_count
    FROM yearly_user_counts
) AS growth_data
ORDER BY year;

 * sqlite:///fetch_sqlmagic.db
Done.


year,user_count,previous_year_count,yoy_growth_percentage
2014,30,None,None
2015,51,30,70.0
2016,70,51,37.25
2017,644,70,820.0
2018,2168,644,236.65
2019,7093,2168,227.17
2020,16883,7093,138.02
2021,19159,16883,13.48
2022,26807,19159,39.92
2023,15464,26807,-42.31
